In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: /opt/ohpc/pub/spack/v0.20.1/opt/spack/linux-rocky8-x86_64_v3/gcc-8.5.0/python-3.10.10-pfj3kn4eqy4hivj33qcmazddy3xivrld/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import os
import tensorflow as tf
import nibabel as nib
import nilearn.image as nl_img
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
import matplotlib.pyplot as plt
import cv2
from nilearn.image import resample_img
from scipy import ndimage
from scipy import interpolate
from scipy.interpolate import RectBivariateSpline
import itertools

2024-04-12 12:37:07.638430: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 12:37:07.645090: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 12:37:07.714916: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 12:37:12.533304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def read_nifti_file(filepath):
    slices = []
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    sagittal = scan.shape[2]
    lower_quartile = int(sagittal * 0.25)
    upper_quartile = int(sagittal * 0.75)
    # Extract slices within the interquartile range
    for i in range(lower_quartile, upper_quartile):
        slice_img = (scan[:, :, i])
        slices.append(slice_img.astype(np.float32))
    return slices

In [4]:
def reshape_image(img):
    # Define the new shape
    new_shape = (256, 256)
    # Define x and y coordinates for original and new shape
    x_old = np.linspace(0, 1, img.shape[1])
    y_old = np.linspace(0, 1, img.shape[0])
    x_new = np.linspace(0, 1, new_shape[1])
    y_new = np.linspace(0, 1, new_shape[0])

    interp_func = RectBivariateSpline(y_old, x_old, img, kx=1, ky=1)

    # Perform interpolation to get the reshaped slice
    reshaped_slice = interp_func(y_new, x_new)
    
    reshaped_slice = np.expand_dims(reshaped_slice, axis=-1)  # Adding channel dimension
#     reshaped_slice = np.expand_dims(reshaped_slice, axis=0)
    return reshaped_slice

In [5]:
# Function to read and preprocess MRI data
def process_data(data_dir):
    final_data = []
    file_paths = [
        os.path.join(data_dir, x)
        for x in os.listdir(data_dir)
        if os.path.isfile(os.path.join(data_dir, x)) and x.endswith('.nii')
    ]
    mri_data = [read_nifti_file(path) for path in file_paths]
    mri_data = list(itertools.chain(*mri_data))
    mri_data = [reshape_image(img) for img in mri_data]
    return mri_data

In [6]:
ad_dir = "ADNI/AD"
cn_dir = "ADNI/CN"
mci_dir = "ADNI/MCI"

# Process MRI data for each class
ad_mri = process_data(ad_dir)
cn_mri = process_data(cn_dir)
mci_mri = process_data(mci_dir)

print(len(ad_mri))
print(len(cn_mri))
print(len(mci_mri))

6261
10270
11651


In [ ]:
print(len(ad_mri))
print(len(cn_mri))
print(len(mci_mri))

test_ad = []
test_cn = []
test_mci = []
while (len(ad_mri) > 5000):
    test_ad.append(ad_mri.pop())
    
while (len(cn_mri) > 5000):
    while (len(test_cn) < len(test_ad)):
        test_cn.append(cn_mri.pop())
    cn_mri.pop()
        
while (len(mci_mri) > 5000):
    while (len(test_mci) < len(test_ad)):
        test_mci.append(mci_mri.pop())
    mci_mri.pop()

print(len(ad_mri))
print(len(cn_mri))
print(len(mci_mri))

print(len(test_ad))
print(len(test_cn))
print(len(test_mci))

In [ ]:
# Define the CNN model
def create_cnn(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3, activation='softmax')  # 3 classes: AD, MCI, CN
    ])
    return model

In [ ]:
# Define input shape (shape of the MRI slices)
input_shape = (256, 256, 1)  # Assuming all MRI slices have the same shape

# batch_size = 32

model = create_cnn(input_shape)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Prepare data and labels
X_train = ad_mri + cn_mri + mci_mri
y_train = [0] * len(ad_mri) + [1] * len(cn_mri) + [2] * len(mci_mri)  # AD: 0, CN: 1, MCI: 2
# X_train = ad_mri + cn_mri
# y_train = [0] * len(ad_mri) + [1] * len(cn_mri)  # AD: 0, CN: 1, MCI: 2
# Convert data and labels to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Shuffle data
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
y_train = y_train[indices]

In [ ]:
# Train the model and record metrics
history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

# Plot learning curve
plt.figure(figsize=(10, 6))

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
X_test = test_ad + test_cn + test_mci
y_test = [0] * len(test_ad) + [1] * len(test_cn) + [2] * len(test_mci)

X_test = np.array(X_test)
y_test = np.array(y_test)

test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)